# Projekt: Web-Scraping von einer realen Webseite

## Ziel des Projekts: 
Ich möchte in diesem Projekt eine Tabelle von einer Wikipedia-Seite nehmen und diese in ein Pandas DataFrame konvertieren. 
Zum Schluss möchte ich das DataFrame abspeichern und in einer CSV-Datei auf meinem lokalen Computer ablegen

## Vorbereitung: Import der benötigten Packages:
- BeautifulSoup wird genutzt um HTML zu parsen <br>
- Pandas um ein später ein DataFrame zu erstellen<br>
- Requests um Webseiten aufzurufen<br>

In [295]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

## Schritt 1: Tabellenüberschriften in ein DataFrame übertragen
Als nächstes definiere ich die URL der Webseite, von der ich die Tabelle nehmen und in ein DataFrame konvertieren möchte. <br>
Anschließend öffne ich die Webseite mir "request" <br>
Und parse den HTML Code mithilfe von BeautifulSoup. 

In [298]:
url = "https://en.wikipedia.org/wiki/List_of_largest_companies_in_the_United_States_by_revenue"
website = requests.get(url)
soup = BeautifulSoup(website.text, "html")

Nun suche ich mithilfe den Entwicklertools auf der oben genannten URL nach dem Namen der Tabelle ("Table Class = "). <br>
Anschließend überprüfe ich mithilfe des "find" Befehls, ob ich tatsächlich die gewünschte Tabelle gefunden habe. 

In [330]:
# soup.find("table", class_ = "wikitable sortable")

In diesem Beispiel gibt es mehrere Tabellen auf der Webseite, die den gleichen Namen "wikitable sortable" haben. <br>
Deswegen definiere ich die gewünschte Tabelle als neue Variable und lege mithilfe eines Index fest, dass ich die erste Tabelle [0] bearbeiten möchte

In [332]:
table = soup.find_all("table")[0]
# print(table)

Als nächstes möchte ich die Tabellenüberschriften meiner Tabelle raussuchen und in eine neue Variable speichern. <br>
Dies lässt sich einfach über die in HTML definierten "Table Header" bzw. "th" lösen.<br> 
Die rohen Tabellenüberschriften müssen anschließend noch bereinigt werden.

In [334]:
table_header_raw = table.find_all("th")
# print(table_header)

In [336]:
table_header = [header.text.strip() for header in table_header_raw]
# print(table_header)

Ich erhalte nun eine Liste mit den jeweiligen Tabellenüberschriften. <br>
Diese kann ich nun in das DataFrame übertragen. 
Als Ergebnis erhalte ich ein leeres DataFrame mit Überschriften.

In [308]:
df = pd.DataFrame(columns = world_table_titles)
df

,Rank,Name,Industry,Revenue (USD millions),Revenue growth,Employees,Headquarters


## Schritt 2: Das Dataframe mit den jewieligen Einträgen vervollständigen: 
So wie ich die Überschriften mit den Tabellenüberschriften herausgefiltert habe, kann ich die einzelnen Zeilen der Tabelle mit "Table Rows" bzw. "tr" herausfiltern. 
Ich möchte hier auch wieder eine Liste erhalten, die ich in einer Variable abspeichere. 
Mit dem nachfolgenden Code erhalte ich eine Liste in der jeder Listeneintrag von "tr" bis "/tr" reicht und somit einer Zeile der Tabelle entspricht.

In [338]:
table_rows = table.find_all("tr")
# print(column_data)

Nun kann ich wieder mithilfe einer Schleife die Gesamte Liste mit den Zeilen (wobei jedes Listenelement einer Zeile in der Tabelle entspricht) durchgehen und so eine neue Liste erstellen (raw_row_data) in der im ersten Durchlauf jedes Listenelement einem einzelnen Eintrag der ersten Zeile entspricht usw. Die einzelnen Tabelleneinträge lassen sich mit "table data" bzw. "td" identifizieren.<br>
<br>
Die Listen, die in "raw_row_data" erstellt werden beinhalten noch HTML code, der in der Nachfolgenden Schleife bereinigt wird und in "row_data" abgespeichert wird.<br>
<br>
Da die Schleife die Listen immer wieder überspeichert, bis sie am Ende angekommen ist, kann ich in den Schleifen-Befehl hinzufügen, dass nach jedem Durchlauf die row_data im DataFrame abgespeichert wird. Dies kann ich mithilfe der len-, und loc-Funktion lösen. 

In [316]:
for row in table_rows[1:]:
    raw_row_data = row.find_all("td")
    row_data = [data.text.strip() for data in raw_row_data]

    lenght = len(df)
    df.loc[lenght] = row_data

In [340]:
# Zum besseren Nachvollziehen: 
print(raw_row_data)
print(row_data)

[<td>100
</td>, <td><a href="/wiki/Best_Buy" title="Best Buy">Best Buy</a>
</td>, <td>Retail
</td>, <td style="text-align:center;">43,452
</td>, <td style="text-align:center;"><span typeof="mw:File"><span title="Decrease"><img alt="Decrease" class="mw-file-element" data-file-height="300" data-file-width="300" decoding="async" height="11" src="//upload.wikimedia.org/wikipedia/commons/thumb/e/ed/Decrease2.svg/11px-Decrease2.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/e/ed/Decrease2.svg/17px-Decrease2.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/e/ed/Decrease2.svg/22px-Decrease2.svg.png 2x" width="11"/></span></span> <span data-sort-value="2999450000000000000♠" style="display:none"></span> -6.1%
</td>, <td style="text-align:center;">85,000
</td>, <td><a href="/wiki/Richfield,_Minnesota" title="Richfield, Minnesota">Richfield, Minnesota</a>
</td>]
['100', 'Best Buy', 'Retail', '43,452', '-6.1%', '85,000', 'Richfield, Minnesota']


## Schritt 3: Finalisierung und Abspeichern als CSV-Datei
Ich erhalte somit folgendes DataFrame:

In [342]:
df.head()

,Rank,Name,Industry,Revenue (USD millions),Revenue growth,Employees,Headquarters
0,1,Walmart,Retail,"648,125",6.0%,"2,100,000","Bentonville, Arkansas"
1,2,Amazon,Retail and cloud computing,"574,785",11.9%,"1,525,000","Seattle, Washington"
2,3,Apple,Electronics industry,"383,482",-2.8%,"161,000","Cupertino, California"
3,4,UnitedHealth Group,Healthcare,"371,622",14.6%,"440,000","Minnetonka, Minnesota"
4,5,Berkshire Hathaway,Conglomerate,"364,482",20.7%,"396,500","Omaha, Nebraska"


Anschließend kann ich das DataFrame lokal auf meinem Computer speichern. Dabei möchte ich den Index des DataFrames nicht in die CSV-Datei übernehmen, weswegen ich noch die Anmerkung "index = False" hinzufüge

In [328]:
df.to_csv(r"C:\Users\denni\Desktop\Data_Science_Projects\Python_Projects\Web_Scraping_Unternehmen\Unternehmen.csv", index = False)